# LDED Audiovisual Fusion 

Author: Chen Lequn.
Created on 13 Sep 2023.

- Material: Maraging Steel 300
- Process: Robotic Llser-directed energy deposition
- Recorded data: position, veolocity, coaxial ccd features, acoustic feature
- Quality labels generated: keyhole pores, cracks, defect-free

### Notebook 2: Feature extraction
- Extract handcrafted features from video and audio stream
- Vision features: melt pool geometric features, including width, length, moment of area, convex hull, etc.
- Audio features: spectral centroid, spectral bandwidth, flux, etc.

### System setup

In [55]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

import os
# Scikit learn
#from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle, resample, class_weight
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from collections import defaultdict

## plot
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
%matplotlib inline
import seaborn as sns

In [56]:
import librosa
import essentia.standard as es
from essentia.standard import Spectrum, Windowing, SpectralCentroidTime, SpectralComplexity, SpectralContrast
from essentia.standard import Decrease, Energy, EnergyBandRatio, FlatnessDB, Flux, RollOff, StrongPeak, CentralMoments
from essentia.standard import DistributionShape, Crest, MelBands, MFCC
import soundfile as sf  # for reading audio files

https://essentia.upf.edu/algorithms_reference.html

In [57]:
PROJECT_ROOT_DIR = "../"
IMAGE_PATH = os.path.join(PROJECT_ROOT_DIR, "result_images", 'feature_extraction')
os.makedirs(IMAGE_PATH, exist_ok=True)

Multimodal_dataset_PATH = "/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset"
Dataset_path = os.path.join(Multimodal_dataset_PATH, f'25Hz_buffered')
                            

## function for automatically save the diagram/graph into the folder 
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGE_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 2.50

In [68]:
def get_sample_directories(base_path, sample_numbers):
    sample_directories = []
    for sample_number in sample_numbers:
        sample_directories.append(os.path.join(base_path, f'{sample_number}'))
    return sample_directories


samples = [21, 22, 23]
sample_directories = get_sample_directories(Dataset_path, samples)

# Get lists of image and audio directories for each sample
image_directories = [os.path.join(sample_dir, 'images') for sample_dir in sample_directories]
audio_directories = [os.path.join(sample_dir, 'raw_audio') for sample_dir in sample_directories]

In [59]:
sample_directories

['/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/21',
 '/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/22',
 '/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/23']

In [89]:
# Combine all annotation files into one DataFrame
all_annotation_dfs = []
for sample_dir, sample_number in zip(sample_directories, samples):
    annotation_file = os.path.join(sample_dir, f'annotations_{sample_number}.csv')  # Update the file name
    annotation_df = pd.read_csv(annotation_file)
    all_annotation_dfs.append(annotation_df)
combined_annotation_df = pd.concat(all_annotation_dfs)
combined_annotation_df

sample index      audio_file_name      image_file_name   class_name  \
0                 1      sample_21_1.wav      sample_21_1.jpg    Laser-off   
1                 2      sample_21_2.wav      sample_21_2.jpg        Crack   
2                 3      sample_21_3.wav      sample_21_3.jpg        Crack   
3                 4      sample_21_4.wav      sample_21_4.jpg        Crack   
4                 5      sample_21_5.wav      sample_21_5.jpg  Defect-free   
...             ...                  ...                  ...          ...   
11391         11392  sample_23_11392.wav  sample_23_11392.jpg    Laser-off   
11392         11393  sample_23_11393.wav  sample_23_11393.jpg    Laser-off   
11393         11394  sample_23_11394.wav  sample_23_11394.jpg    Laser-off   
11394         11395  sample_23_11395.wav  sample_23_11395.jpg    Laser-off   
11395         11396  sample_23_11396.wav  sample_23_11396.jpg    Laser-off   

      class_name_v2  Layer number  Sample number         X         Y         Z  
0         Laser-off           1.0             21  0.136620  0.265042  9.588183  
1         Defective           1.0             21  0.128219  0.275541  3.966295  
2         Defective           1.0             21  0.126358  0.276802  2.995122  
3         Defective           1.0             21  0.124890  0.278469  2.108443  
4       Defect-free           1.0             21  0.124157  0.278818  1.804407  
...             ...           ...            ...       ...       ...       ...  
11391     Laser-off          50.0             23       NaN       NaN       NaN  
11392     Laser-off          50.0             23       NaN       NaN       NaN  
11393     Laser-off          50.0             23       NaN       NaN       NaN  
11394     Laser-off          50.0             23       NaN       NaN       NaN  
11395     Laser-off          50.0             23       NaN       NaN       NaN  

[21161 rows x 10 columns]

## Extracting melt pool visual features

In [61]:
def general_contour_extraction(image, threshold=100):
    """
    Extract general contour features from a given image.
    
    Parameters:
        image (ndarray): The input image.
        threshold (int): The threshold value for image processing.
    
    Returns:
        dict: A dictionary containing the extracted features.
    """
    # Initialize the result dictionary with zeros
    result = {
        'max_contour_area': 0,
        'rectangle_angle': 0,
        'rectangle_width': 0,
        'rectangle_height': 0,
        'ellipse_angle': 0,
        'ellipse_width': 0,
        'ellipse_height': 0
    }
    
    # Convert the image to grayscale
    src_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply blur
    src_gray = cv2.blur(src_gray, (3, 3))
    
    # Apply threshold
    _, threshold_output = cv2.threshold(src_gray, threshold, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(threshold_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        return result  # Return result with zeros if no contours are found
    
    # Find the rotated rectangles and ellipses for each contour
    min_rects = [cv2.minAreaRect(np.array(contour)) for contour in contours]
    contour_areas = [cv2.contourArea(np.array(contour)) for contour in contours]
    
    # Get the index of the max contour area
    max_contour_area_index = np.argmax(contour_areas)
    max_contour_area = contour_areas[max_contour_area_index]
    
    # Store the max contour area
    result['max_contour_area'] = max_contour_area
    
    # Store rectangle features
    rect = min_rects[max_contour_area_index]
    result['rectangle_angle'] = rect[-1]
    result['rectangle_width'] = rect[1][0]
    result['rectangle_height'] = rect[1][1]
    
    # Store ellipse features if enough points for fitEllipse
    if len(contours[max_contour_area_index]) > 5:
        ellipse = cv2.fitEllipse(np.array(contours[max_contour_area_index]))
        result['ellipse_angle'] = ellipse[-1]
        result['ellipse_width'] = ellipse[1][0]
        result['ellipse_height'] = ellipse[1][1]
    
    return result

In [22]:
def convex_hull_extract(frame, threshold=100):
    """
    Extract convex hull features from a given image.
    
    Parameters:
        image_path (str): The path to the image file.
        threshold (int): The threshold value for binary conversion.
    
    Returns:
        max_hull_area (float): The maximum area among all convex hulls.
    """
    
    # Convert to grayscale if the image is colored
    if frame.shape[-1] > 1:
        src_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    else:
        src_gray = frame

    # Blur the image
    src_gray = cv2.blur(src_gray, (3, 3))
    
    # Apply threshold
    ret, threshold_output = cv2.threshold(src_gray, threshold, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(threshold_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize return values
    max_hull_area = 0.0

    # Check if any contour is detected
    if contours:
        # Find the convex hull object for each contour
        hull = [cv2.convexHull(cnt) for cnt in contours]
        
        # Find the bounding convex hull area for each contour
        hull_area = [cv2.contourArea(h) for h in hull]
        
        # Get the maximum convex hull area
        max_hull_area = max(hull_area)
        
#         # Draw contours and convex hull on the original image (for visualization)
#         drawing = np.zeros((threshold_output.shape[0], threshold_output.shape[1], 3), dtype=np.uint8)
#         for i in range(len(contours)):
#             color = (np.random.randint(0,256), np.random.randint(0,256), np.random.randint(0,256))
#             cv2.drawContours(drawing, contours, i, color)
#             cv2.drawContours(drawing, hull, i, color, 2)
        
#         # Show the output image with contours and convex hull
#         plt.imshow(cv2.cvtColor(drawing, cv2.COLOR_BGR2RGB))
#         plt.title('Contours and Convex Hull')
#         plt.axis('off')
#         plt.show()
        
    return max_hull_area

In [23]:
# Feature extraction for moments
def moment_extract(image, threshold):
    # Initialize moments as zeros
    features = {
        'm00': 0,
        'm10': 0,
        'm01': 0,
        'm20': 0,
        'm11': 0,
        'm02': 0,
        'm30': 0,
        'm21': 0,
        'm12': 0,
        'm03': 0,
        'mu20': 0,
        'mu11': 0,
        'mu02': 0,
        'mu30': 0,
        'mu21': 0,
        'mu12': 0,
        'mu03': 0,
        'nu20': 0,
        'nu11': 0,
        'nu02': 0,
        'nu30': 0,
        'nu21': 0,
        'nu12': 0,
        'nu03': 0,
        'center_x': 0,
        'center_y': 0,
        'contour_area': 0,
        'contour_length': 0
    }
    
    # Convert to grayscale if the image is colored
    if len(image.shape) > 2:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        gray = image

    # Thresholding
    _, thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # Sort contours by area
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        moments = cv2.moments(largest_contour)
        
        # Avoid division by zero
        if moments['m00'] != 0:
            for moment_name, moment_value in moments.items():
                features[moment_name] = moment_value
                
            features['center_x'] = moments['m10'] / moments['m00']
            features['center_y'] = moments['m01'] / moments['m00']
            features['contour_area'] = cv2.contourArea(largest_contour)
            features['contour_length'] = cv2.arcLength(largest_contour, True)
            
    return features

### Extract all visual features

In [67]:
audio_directories

(['/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/21/raw_audio',
  '/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/22/raw_audio',
  '/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/23/raw_audio'],
 26)

In [63]:
def extract_visual_features(image_directories, threshold=100):
    all_features_list = []
    total_images = sum([len(os.listdir(img_dir)) for img_dir in image_directories if os.path.isdir(img_dir)])
    pbar = tqdm(total=total_images, desc="Processing images")

    for img_dir in image_directories:
        if os.path.isdir(img_dir):
            for img_name in os.listdir(img_dir):
                if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                    img_path = os.path.join(img_dir, img_name)
                    img = cv2.imread(img_path)
                    
                    features_contour = general_contour_extraction(img, threshold=threshold)
                    max_hull = convex_hull_extract(img, threshold=threshold)
                    features_moments = moment_extract(img, threshold=threshold)
                    
                    # Merge all dictionaries into one
                    merged_features = {'image_file_name': img_name, **features_contour, 'max_hull': max_hull, **features_moments}
                    all_features_list.append(merged_features)
                    
                    pbar.update(1)
    
    pbar.close()
    return pd.DataFrame(all_features_list)

In [64]:
df_visual = extract_visual_features(image_directories)
df_visual.head()

Processing images: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21163/21163 [01:50<00:00, 192.06it/s]


image_file_name  max_contour_area  rectangle_angle  rectangle_width  \
0  sample_21_2038.jpg          231979.0             90.0       479.000000   
1  sample_21_2716.jpg          247762.5             90.0       451.000000   
2  sample_21_2351.jpg          214554.0             90.0       479.000000   
3  sample_21_1539.jpg          192581.0             90.0       479.000000   
4  sample_21_4902.jpg          258557.5              0.0       565.999878   

   rectangle_height  ellipse_angle  ellipse_width  ellipse_height  max_hull  \
0        528.000000       2.577053     574.159973      818.167725  236628.0   
1        639.000000      81.376549     537.173401      825.702881  260181.0   
2        501.000000       3.983349     522.245422      614.861877  219719.5   
3        475.000000       4.050855     467.903259      567.466492  200381.0   
4        478.999939     178.969452     587.517456      886.574585  263607.5   

        m00  ...      nu11      nu02      nu30      nu21      nu12      nu03  \
0  232057.0  ... -0.006111  0.078487  0.000851 -0.002744 -0.000768  0.001435   
1  247724.0  ... -0.007437  0.053936  0.004801 -0.003457 -0.001749  0.001352   
2  214637.0  ... -0.001850  0.081376  0.000582 -0.000139 -0.000970 -0.000040   
3  192548.0  ... -0.001962  0.086579  0.000307  0.000450 -0.000491 -0.000814   
4  258438.0  ... -0.002776  0.072466  0.000320 -0.000871 -0.000617  0.000196   

     center_x    center_y  contour_area  contour_length  
0  247.358734  229.456223      232057.0     2326.891477  
1  309.548025  195.965763      247724.0     2372.048770  
2  234.949085  237.435629      214637.0     2272.714850  
3  232.527353  238.605859      192548.0     2438.444862  
4  270.647355  236.184484      258438.0     2342.783832  

[5 rows x 37 columns]

## Extract Audio Features

In [70]:
audio_path = os.path.join(audio_directories[1], "sample_22_9.wav")
audio_signal, sample_rate = sf.read(audio_path, dtype='float32')
# print(sample_rate)
# print (len(audio_signal))
# print (len(audio_signal)/sample_rate)
# plt.plot(audio_signal)

In [71]:
def check_audio_lengths(audio_file_paths):
    length_dict = defaultdict(list)
    
    for file_path in audio_file_paths:
        audio_signal, sr = librosa.load(file_path, sr=None)
        length_in_seconds = len(audio_signal) / sr
        length_dict[length_in_seconds].append(file_path)
        
    if len(length_dict) == 1:
        print(f"All audio files have the same length: {list(length_dict.keys())[0]} seconds.")
        return True
    else:
        print("Not all audio files have the same length.")
        for length, files in length_dict.items():
            print(f"Length: {length} seconds -> Files: {files}")
        return False

In [72]:
import os

def example_usage_check_audio_lengths(audio_directories):
    # Initialize an empty list to store audio file paths
    audio_file_paths = []
    
    # Iterate over each directory in audio_directories to collect audio file paths
    for directory in audio_directories:
        for file_name in os.listdir(directory):
            if file_name.endswith(".wav"):
                audio_file_paths.append(os.path.join(directory, file_name))
    
    # Call the check_audio_lengths function
    return check_audio_lengths(audio_file_paths)


# Uncomment the line below to run the function
example_usage_check_audio_lengths(audio_directories)

All audio files have the same length: 0.1 seconds.


True

In [13]:
def extract_time_domain_features(audio_signal, sample_rate=44100):
    """
    Extract time domain features from an audio signal using Essentia.
    
    Parameters:
    - audio_signal: numpy array, the audio signal from which to extract features
    - sample_rate: int, the sample rate of the audio signal
    
    Returns:
    - features: dict, a dictionary containing the extracted features
    """
    
    features = {}
    
    # RMS Energy
    rms_algo = es.RMS()
    rms_energy = rms_algo(audio_signal)
    features['rms_energy'] = rms_energy
    
    # Amplitude Envelope
    envelope_algo = es.Envelope()
    amplitude_envelope = envelope_algo(audio_signal)
    features['amplitude_envelope_mean'] = amplitude_envelope.mean()
    features['amplitude_envelope_std'] = amplitude_envelope.std()
    
    # Zero Crossing Rate
    zcr_algo = es.ZeroCrossingRate()
    zero_crossing_rate = zcr_algo(audio_signal)
    features['zero_crossing_rate'] = zero_crossing_rate
    
    # Dynamic Complexity and Loudness
    dyn_algo = es.DynamicComplexity()
    dynamic_complexity, loudness = dyn_algo(audio_signal)
    features['dynamic_complexity'] = dynamic_complexity
    features['loudness'] = loudness

    # Loudness Vickers
    loudness_algo = es.LoudnessVickers()
    loudness_vickers = loudness_algo(audio_signal)
    features['loudness_vickers'] = loudness_vickers

    return features

Essentia provides a variety of spectral descriptors that you can use for feature extraction:

1. **Spectral Centroid**: Computes the center of mass of the spectrum.
2. **Spectral Complexity**: Measures the amount of peak-like components in the spectrum.
3. **Spectral Contrast**: Computes the spectral contrast features from an audio signal.
4. **Spectral Decrease**: Computes the decrease of the spectrum.
5. **Spectral Energy**: Computes the energy of the frequency domain signal.
6. **Spectral Energy Band Ratio**: Computes the ratio of energy in specific bands to the total energy.
7. **Spectral Flatness**: Computes the flatness of a spectrum.
8. **Spectral Flux**: Computes the flux of the spectrum.
9. **Spectral Rolloff**: Computes the rolloff frequency of an audio signal.
10. **Spectral Strong Peak**: Computes the strong peak of the spectrum.
12. **Spectral Variance, skewness, kurtosis**: Computes the variance of the spectral peaks.
14. **MFCC (Mel Frequency Cepstral Coefficients)**: Widely used spectral feature in audio and speech processing.


In [38]:
def extract_spectral_features(audio_signal, sample_rate, frame_size=1024, hop_size=512):
    # Initialize the algorithms
    window_algo = Windowing(type='hann')
    spectrum_algo = Spectrum()
    centroid_algo = SpectralCentroidTime(sampleRate=sample_rate)
    complexity_algo = SpectralComplexity(sampleRate=sample_rate)
    contrast_algo = SpectralContrast(frameSize=frame_size, highFrequencyBound=sample_rate/2, lowFrequencyBound=200, sampleRate=sample_rate)
    decrease_algo = Decrease()
    energy_algo = Energy()
    energy_band_ratio_algo = EnergyBandRatio(sampleRate=sample_rate, stopFrequency=7000)
    flatness_algo = FlatnessDB()
    spectral_flux = Flux()
    rolloff_algo = RollOff(sampleRate=sample_rate)
    strong_peak_algo = StrongPeak()
    central_moment_algo = CentralMoments()
    distrubution_shape = DistributionShape()
    spectral_crest_factor = Crest()
    mel_bands_algo = MelBands()
    mfcc_algo = MFCC(inputSize=hop_size+1, highFrequencyBound=sample_rate/2, numberCoefficients=13, sampleRate=sample_rate)
    
    # Initialize features dictionary with defaultdict to store lists
    # features = {}
    features = defaultdict(list)
    
    for frame in es.FrameGenerator(audio_signal, frameSize=frame_size, hopSize=hop_size):
        windowed_frame = window_algo(frame)
        spectrum = spectrum_algo(windowed_frame)

        features['spectral_centroid'].append(centroid_algo(spectrum))
        features['spectral_complexity'].append(complexity_algo(spectrum))
        spectral_contrast, spectral_valley = contrast_algo(spectrum)
        for i, val in enumerate(spectral_contrast):
            features[f'spectral_contrast_{i}'].append(val)
        for i, val in enumerate(spectral_valley):
            features[f'spectral_valley_{i}'].append(val)
        features['spectral_decrease'].append(decrease_algo(spectrum))
        features['spectral_energy'].append(energy_algo(spectrum))
        features['spectral_energy_band_ratio'].append(energy_band_ratio_algo(spectrum))
        features['spectral_flatness'].append(flatness_algo(spectrum))
        features['spectral_flux'].append(spectral_flux(spectrum))
        features['spectral_rolloff'].append(rolloff_algo(spectrum))
        features['spectral_strong_peak'].append(strong_peak_algo(spectrum))
        central_moments = central_moment_algo(spectrum)
        features['spectral_variance'].append(distrubution_shape(central_moments)[0])
        features['spectral_skewness'].append(distrubution_shape(central_moments)[1])
        features['spectral_kurtosis'].append(distrubution_shape(central_moments)[2])
        features['spectral_crest_factor'].append(spectral_crest_factor(spectrum))

        mfcc_bands, mfcc_coeffs = mfcc_algo(spectrum)
        for i, coeff in enumerate(mfcc_coeffs):
            features[f'mfcc_{i}'].append(coeff)
            
    # Prepare a dictionary to store mean and std separately
    features_separated = {}
    for key, value in features.items():
        mean_val = np.mean(value)
        std_val = np.std(value)
        features_separated[f"{key}_mean"] = mean_val
        features_separated[f"{key}_std"] = std_val
    
    return features_separated

In [39]:
# Example usage
sample_rate = 44100
audio_signal = np.random.rand(4410).astype(np.float32)  
features = extract_spectral_features(audio_signal, sample_rate, frame_size=1024)
features

{'spectral_centroid_mean': 6270.316479492188,
 'spectral_centroid_std': 2034.9482012011142,
 'spectral_complexity_mean': 29.1,
 'spectral_complexity_std': 6.024118192731613,
 'spectral_contrast_0_mean': -0.7685884,
 'spectral_contrast_0_std': 0.035475347,
 'spectral_contrast_1_mean': -0.7829323,
 'spectral_contrast_1_std': 0.03556331,
 'spectral_contrast_2_mean': -0.7858219,
 'spectral_contrast_2_std': 0.026128374,
 'spectral_contrast_3_mean': -0.7752096,
 'spectral_contrast_3_std': 0.020860018,
 'spectral_contrast_4_mean': -0.7666172,
 'spectral_contrast_4_std': 0.03136995,
 'spectral_contrast_5_mean': -0.7576519,
 'spectral_contrast_5_std': 0.019240327,
 'spectral_valley_0_mean': -4.4855905,
 'spectral_valley_0_std': 0.24730648,
 'spectral_valley_1_mean': -4.691103,
 'spectral_valley_1_std': 0.4366002,
 'spectral_valley_2_mean': -4.6499915,
 'spectral_valley_2_std': 0.3523167,
 'spectral_valley_3_mean': -4.7970243,
 'spectral_valley_3_std': 0.36294726,
 'spectral_valley_4_mean': -4.7

### Extract all audio features

In [73]:
def extract_all_audio_features(audio_directories, frame_size=1024, hop_size=512):
    all_features_list = []
    
    # Count total audio files for progress bar
    total_audio_files = sum([len(os.listdir(audio_dir)) for audio_dir in audio_directories if os.path.isdir(audio_dir)])
    
    pbar = tqdm(total=total_audio_files, desc="Processing audio files")

    for audio_dir in audio_directories:
        if os.path.isdir(audio_dir):
            for audio_name in os.listdir(audio_dir):
                if audio_name.lower().endswith(('.wav', '.flac', '.mp3')):
                    audio_path = os.path.join(audio_dir, audio_name)
                    
                    # Read audio file
                    audio_signal, sample_rate = sf.read(audio_path, dtype='float32')
                    
                    # Extract features
                    time_domain_features = extract_time_domain_features(audio_signal, sample_rate)
                    spectral_features = extract_spectral_features(audio_signal, sample_rate, frame_size, hop_size)
                    
                    # Merge all dictionaries into one
                    merged_features = {'audio_file_name': audio_name, **time_domain_features, **spectral_features}
                    all_features_list.append(merged_features)
                    
                    pbar.update(1)
    
    pbar.close()
    return pd.DataFrame(all_features_list)


In [74]:
audio_directories

['/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/21/raw_audio',
 '/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/22/raw_audio',
 '/home/chenlequn/Dataset/LDED_acoustic_visual_monitoring_dataset/25Hz_buffered/23/raw_audio']

In [75]:
time_domain_features = extract_time_domain_features(audio_signal, sample_rate)
time_domain_features

{'rms_energy': 0.03374376893043518,
 'amplitude_envelope_mean': 0.03788913,
 'amplitude_envelope_std': 0.013862286,
 'zero_crossing_rate': 0.07800453156232834,
 'dynamic_complexity': 0.0,
 'loudness': -100.0,
 'loudness_vickers': -37.499725341796875}

In [76]:
audio_features_df = extract_all_audio_features(audio_directories, frame_size=1024, hop_size=512)

Processing audio files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21163/21163 [02:07<00:00, 166.60it/s]


## Save extracted features

In [77]:
audio_features_df

audio_file_name  rms_energy  amplitude_envelope_mean  \
0       sample_21_1833.wav    0.025188                 0.028686   
1       sample_21_1023.wav    0.028820                 0.030098   
2       sample_21_2382.wav    0.027742                 0.028501   
3       sample_21_4248.wav    0.034371                 0.042144   
4       sample_21_2127.wav    0.028227                 0.029618   
...                    ...         ...                      ...   
21158   sample_23_1588.wav    0.023617                 0.026640   
21159  sample_23_11310.wav    0.015761                 0.018173   
21160   sample_23_7809.wav    0.014604                 0.015627   
21161   sample_23_3553.wav    0.010549                 0.011206   
21162   sample_23_6536.wav    0.015022                 0.017994   

       amplitude_envelope_std  zero_crossing_rate  dynamic_complexity  \
0                    0.010267            0.103175                 0.0   
1                    0.013072            0.128345                 0.0   
2                    0.013807            0.107483                 0.0   
3                    0.014231            0.158503                 0.0   
4                    0.012488            0.075283                 0.0   
...                       ...                 ...                 ...   
21158                0.009514            0.162812                 0.0   
21159                0.006428            0.176644                 0.0   
21160                0.005369            0.113832                 0.0   
21161                0.004722            0.119274                 0.0   
21162                0.004684            0.108844                 0.0   

       loudness  loudness_vickers  spectral_centroid_mean  \
0        -100.0        -39.429451             4358.164465   
1        -100.0        -37.971367             4577.742078   
2        -100.0        -38.194988             4318.027124   
3        -100.0        -33.375977             3936.613757   
4        -100.0        -38.530449             5076.497595   
...         ...               ...                     ...   
21158    -100.0        -37.063110             4228.148169   
21159    -100.0        -41.150414             4226.324109   
21160    -100.0        -43.164276             3964.088940   
21161    -100.0        -44.758076             4009.618140   
21162    -100.0        -42.593674             3594.629822   

       spectral_centroid_std  ...  mfcc_8_mean  mfcc_8_std  mfcc_9_mean  \
0                1707.299206  ...     7.795312    8.760357    -4.066066   
1                1522.918029  ...    16.385172    7.940246    -2.424395   
2                1397.846579  ...    13.058914    4.139112     0.432728   
3                1103.515635  ...    17.916513    4.974156    -6.706927   
4                1857.173434  ...    14.958076    4.501075    -2.989413   
...                      ...  ...          ...         ...          ...   
21158            1514.605615  ...    17.289114    4.993794     6.978894   
21159            1230.620592  ...    27.976917    6.414967     7.879941   
21160            1447.116651  ...    15.674917    4.910372    14.549416   
21161            1366.597497  ...     6.898334    6.906912     1.399442   
21162            1342.564882  ...    14.821383    7.172950    19.407732   

       mfcc_9_std  mfcc_10_mean  mfcc_10_std  mfcc_11_mean  mfcc_11_std  \
0        4.982888      4.512198     3.749615     20.473537     4.237676   
1        7.367054      6.193217     5.281717     22.323563     5.212643   
2        5.368427      3.747229     4.530239      8.794775     4.628968   
3        6.211168      3.747542     4.504618     12.932431     4.797512   
4        6.751957     10.608861     6.124709      3.756908     4.412839   
...           ...           ...          ...           ...          ...   
21158   10.670244      0.601029     5.406055      6.375392     5.140900   
21159    6.459905      4.421271     7.511581     17.965603     6.006993   
21160    6.223956   

In [78]:
df_visual

image_file_name  max_contour_area  rectangle_angle  \
0       sample_21_2038.jpg          231979.0             90.0   
1       sample_21_2716.jpg          247762.5             90.0   
2       sample_21_2351.jpg          214554.0             90.0   
3       sample_21_1539.jpg          192581.0             90.0   
4       sample_21_4902.jpg          258557.5              0.0   
...                    ...               ...              ...   
21158   sample_23_4635.jpg               0.0              0.0   
21159   sample_23_3089.jpg               0.0              0.0   
21160   sample_23_9146.jpg               0.0              0.0   
21161  sample_23_10913.jpg          174897.5             -0.0   
21162   sample_23_2098.jpg          300587.0              0.0   

       rectangle_width  rectangle_height  ellipse_angle  ellipse_width  \
0           479.000000        528.000000       2.577053     574.159973   
1           451.000000        639.000000      81.376549     537.173401   
2           479.000000        501.000000       3.983349     522.245422   
3           479.000000        475.000000       4.050855     467.903259   
4           565.999878        478.999939     178.969452     587.517456   
...                ...               ...            ...            ...   
21158         0.000000          0.000000       0.000000       0.000000   
21159         0.000000          0.000000       0.000000       0.000000   
21160         0.000000          0.000000       0.000000       0.000000   
21161       428.999939        476.999939       4.370568     422.613373   
21162       638.999878        478.999939      92.833946     641.902832   

       ellipse_height  max_hull       m00  ...      nu11      nu02      nu30  \
0          818.167725  236628.0  232057.0  ... -0.006111  0.078487  0.000851   
1          825.702881  260181.0  247724.0  ... -0.007437  0.053936  0.004801   
2          614.861877  219719.5  214637.0  ... -0.001850  0.081376  0.000582   
3          567.466492  200381.0  192548.0  ... -0.001962  0.086579  0.000307   
4          886.574585  263607.5  258438.0  ... -0.002776  0.072466  0.000320   
...               ...       ...       ...  ...       ...       ...       ...   
21158        0.000000       0.0       0.0  ...  0.000000  0.000000  0.000000   
21159        0.000000       0.0       0.0  ...  0.000000  0.000000  0.000000   
21160        0.000000       0.0       0.0  ...  0.000000  0.000000  0.000000   
21161      603.366272  180685.0  174952.0  ... -0.002578  0.095400 -0.000198   
21162      798.648132  302261.5  300882.0  ... -0.003022  0.061993  0.000445   

           nu21      nu12      nu03    center_x    center_y  contour_area  \
0     -0.002744 -0.000768  0.001435  247.358734  229.456223      232057.0   
1     -0.003457 -0.001749  0.001352  309.548025  195.965763      247724.0   
2     -0.000139 -0.000970 -0.000040  234.949085  237.435629      214637.0   
3      0.000450 -0.000491 -0.000814  232.527353  238.605859      192548.0   
4     -0.000871 -0.000617  0.000196  270.647355  236.184484      258438.0   
...         ...       ...       ...         ...         ...           ...   
21158  0.000000  0.000000  0.000000    0.000000    0.000000           0.0   
21159  0.000000  0.000000  0.000000    0.000000    0.000000           0.0   
21160  0.000000  0.000000  0.000000    0.000000    0.000000           0.0   
21161 -0.001218  0.000918  0.000290  236.736220  229.696824      174952.0   
21162 -0.000961 -0.000688  0.000211  315.182824  235.842443      300882.0   

       contour_length  
0         2326.891477  
1         2372.048770  
2         2272.714850  
3         2438.444862  
4         2342.783832  
...               ...  
21158        0.000000  
21159        0.000000  
21160        0.000000  
21161     2008.680361  
21162     2325.480229  

[21163 rows x 37 columns]

In [93]:
df_visual.columns

Index(['image_file_name', 'max_contour_area', 'rectangle_angle',
       'rectangle_width', 'rectangle_height', 'ellipse_angle', 'ellipse_width',
       'ellipse_height', 'max_hull', 'm00', 'm10', 'm01', 'm20', 'm11', 'm02',
       'm30', 'm21', 'm12', 'm03', 'mu20', 'mu11', 'mu02', 'mu30', 'mu21',
       'mu12', 'mu03', 'nu20', 'nu11', 'nu02', 'nu30', 'nu21', 'nu12', 'nu03',
       'center_x', 'center_y', 'contour_area', 'contour_length'],
      dtype='object')

In [94]:
audio_features_df.columns

Index(['audio_file_name', 'rms_energy', 'amplitude_envelope_mean',
       'amplitude_envelope_std', 'zero_crossing_rate', 'dynamic_complexity',
       'loudness', 'loudness_vickers', 'spectral_centroid_mean',
       'spectral_centroid_std', 'spectral_complexity_mean',
       'spectral_complexity_std', 'spectral_contrast_0_mean',
       'spectral_contrast_0_std', 'spectral_contrast_1_mean',
       'spectral_contrast_1_std', 'spectral_contrast_2_mean',
       'spectral_contrast_2_std', 'spectral_contrast_3_mean',
       'spectral_contrast_3_std', 'spectral_contrast_4_mean',
       'spectral_contrast_4_std', 'spectral_contrast_5_mean',
       'spectral_contrast_5_std', 'spectral_valley_0_mean',
       'spectral_valley_0_std', 'spectral_valley_1_mean',
       'spectral_valley_1_std', 'spectral_valley_2_mean',
       'spectral_valley_2_std', 'spectral_valley_3_mean',
       'spectral_valley_3_std', 'spectral_valley_4_mean',
       'spectral_valley_4_std', 'spectral_valley_5_mean',
       's

In [85]:
combined_annotation_df

sample index      audio_file_name      image_file_name         name  \
0                 1      sample_21_1.wav      sample_21_1.jpg    Laser-off   
1                 2      sample_21_2.wav      sample_21_2.jpg        Crack   
2                 3      sample_21_3.wav      sample_21_3.jpg        Crack   
3                 4      sample_21_4.wav      sample_21_4.jpg        Crack   
4                 5      sample_21_5.wav      sample_21_5.jpg  Defect-free   
...             ...                  ...                  ...          ...   
11391         11392  sample_23_11392.wav  sample_23_11392.jpg          NaN   
11392         11393  sample_23_11393.wav  sample_23_11393.jpg          NaN   
11393         11394  sample_23_11394.wav  sample_23_11394.jpg          NaN   
11394         11395  sample_23_11395.wav  sample_23_11395.jpg          NaN   
11395         11396  sample_23_11396.wav  sample_23_11396.jpg          NaN   

       class_ID class_name_v2  class_ID_2  Layer number  Sample number  \
0           0.0     Laser-off         0.0           1.0             21   
1           2.0     Defective         2.0           1.0             21   
2           2.0     Defective         2.0           1.0             21   
3           2.0     Defective         2.0           1.0             21   
4           1.0   Defect-free         1.0           1.0             21   
...         ...           ...         ...           ...            ...   
11391       NaN     Laser-off         NaN          50.0             23   
11392       NaN     Laser-off         NaN          50.0             23   
11393       NaN     Laser-off         NaN          50.0             23   
11394       NaN     Laser-off         NaN          50.0             23   
11395       NaN     Laser-off         NaN          50.0             23   

              X         Y         Z      name2  classification class_name  
0      0.136620  0.265042  9.588183  Laser-off             0.0        NaN  
1      0.128219  0.275541  3.966295     Defect             2.0        NaN  
2      0.126358  0.276802  2.995122     Defect             2.0        NaN  
3      0.124890  0.278469  2.108443     Defect             2.0        NaN  
4      0.124157  0.278818  1.804407     Defect             2.0        NaN  
...         ...       ...       ...        ...             ...        ...  
11391       NaN       NaN       NaN        NaN             NaN  Laser-off  
11392       NaN       NaN       NaN        NaN             NaN  Laser-off  
11393       NaN       NaN       NaN        NaN             NaN  Laser-off  
11394       NaN       NaN       NaN        NaN             NaN  Laser-off  
11395       NaN       NaN       NaN        NaN             NaN  Laser-off  

[21161 rows x 15 columns]

In [90]:
# Merge the annotation dataframe with the audio and visual dataframes
df_audiovisual = combined_annotation_df.merge(audio_features_df, how='left', on='audio_file_name')
df_audiovisual = df_audiovisual.merge(df_visual, how='left', on='image_file_name')

# Show the first few rows of the merged dataframe
df_audiovisual

sample index      audio_file_name      image_file_name   class_name  \
0                 1      sample_21_1.wav      sample_21_1.jpg    Laser-off   
1                 2      sample_21_2.wav      sample_21_2.jpg        Crack   
2                 3      sample_21_3.wav      sample_21_3.jpg        Crack   
3                 4      sample_21_4.wav      sample_21_4.jpg        Crack   
4                 5      sample_21_5.wav      sample_21_5.jpg  Defect-free   
...             ...                  ...                  ...          ...   
21156         11392  sample_23_11392.wav  sample_23_11392.jpg    Laser-off   
21157         11393  sample_23_11393.wav  sample_23_11393.jpg    Laser-off   
21158         11394  sample_23_11394.wav  sample_23_11394.jpg    Laser-off   
21159         11395  sample_23_11395.wav  sample_23_11395.jpg    Laser-off   
21160         11396  sample_23_11396.wav  sample_23_11396.jpg    Laser-off   

      class_name_v2  Layer number  Sample number         X         Y  \
0         Laser-off           1.0             21  0.136620  0.265042   
1         Defective           1.0             21  0.128219  0.275541   
2         Defective           1.0             21  0.126358  0.276802   
3         Defective           1.0             21  0.124890  0.278469   
4       Defect-free           1.0             21  0.124157  0.278818   
...             ...           ...            ...       ...       ...   
21156     Laser-off          50.0             23       NaN       NaN   
21157     Laser-off          50.0             23       NaN       NaN   
21158     Laser-off          50.0             23       NaN       NaN   
21159     Laser-off          50.0             23       NaN       NaN   
21160     Laser-off          50.0             23       NaN       NaN   

              Z  ...      nu11      nu02      nu30      nu21      nu12  \
0      9.588183  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
1      3.966295  ...  0.000000  0.062337  0.000000  0.000000  0.000000   
2      2.995122  ...  0.000000  0.062337  0.000000  0.000000  0.000000   
3      2.108443  ... -0.008049  0.062216  0.001940 -0.002304 -0.001599   
4      1.804407  ... -0.009879  0.065910  0.001738 -0.003345 -0.000898   
...         ...  ...       ...       ...       ...       ...       ...   
21156       NaN  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
21157       NaN  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
21158       NaN  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
21159       NaN  ...  0.000000  0.000000  0.000000  0.000000  0.000000   
21160       NaN  ...  0.000000  0.000000  0.000000  0.000000  0.000000   

           nu03    center_x    center_y  contour_area  contour_length  
0      0.000000    0.000000    0.000000           0.0        0.000000  
1      0.000000  319.500000  239.000000      305442.0     2234.000000  
2      0.000000  319.500000  239.000000      305442.0     2234.000000  
3      0.000891  307.440996  230.264496      291865.5     2279.781744  
4      0.001388  296.636877  229.109962      281970.0     2407.847760  
...         ...         ...         ...           ...             ...  
21156  0.000000    0.000000    0.000000           0.0        0.000000  
21157  0.000000    0.000000    0.000000           0.0        0.000000  
21158  0.000000    0.000000    0.000000           0.0        0.000000  
21159  0.000000    0.000000    0.000000           0.0        0.000000  
21160  0.000000    0.000000    0.000000           0.0        0.000000  

[21161 rows x 129 columns]

In [91]:
for col in ['audio_file_name', 'image_file_name', 'class_name', 'class_name_v2']:
    df_audiovisual[col] = df_audiovisual[col].astype('category')

In [92]:
df_audiovisual.to_hdf(os.path.join(Dataset_path, 'data_audio_visual.h5'), key='df', mode='w', format='table')